# Preamble

In [7]:
# json
import json

# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
from torch import Tensor

import matplotlib.pyplot as plt
import librosa

import librosa.display
import IPython.display as ipd

import numpy as np
import pickle
import random
import torch
import librosa
from torch_specinv import griffin_lim
from torch_specinv.metrics import spectral_convergence as SC

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn import metrics
import youtube_dl
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

import cvxpy as cp
import math
from torchaudio import functional as AF

In [8]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
use_cuda=True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

CUDA Available:  True


In [9]:
FIXED_SAMPLE_RATE = 48000

# Load data

In [32]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from classification.models.DeepRecursiveCNN import DeepRecursiveCNN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
pickle_path = "/nfs/students/summer-term-2020/project-4/data/dataset1/dataset_resampled/"
training = pickle.load(open(pickle_path + "training.p","rb"))
validation = pickle.load(open(pickle_path + "validation.p","rb"))

In [11]:
max_length = 0

for sample in training:
    max_length = max(max_length, sample['data'][0].shape[0])

for sample in validation:
    max_length = max(max_length, sample['data'][0].shape[0])
    
print(max_length)

481489


In [12]:
def prepareData(data):
    zero_padded_data = torch.zeros(max_length)
    zero_padded_data[:data.shape[0]] = torch.from_numpy(data)
    return zero_padded_data

In [13]:
training_dataset = [(prepareData(sample['data'][0]), 1)
                        if sample['binary_class']=='positive' else (prepareData(sample['data'][0]), 0) 
                        for sample in training]
validation_dataset = [(prepareData(sample['data'][0]), 1)
                        if sample['binary_class']=='positive' else (prepareData(sample['data'][0]), 0) 
                        for sample in validation]

## Loading model

In [206]:
model_state_dict_path = "/nfs/students/summer-term-2020/project-4/yan/models/best_model_state_dict.pt"
model = DeepRecursiveCNN()
model.load_state_dict(torch.load(model_state_dict_path))
model = model.eval()

# Attacks

## New attack

In [15]:
## speedup with padding
def time_stretch(sample, speedup_rate):
    if speedup_rate == 1:
        return sample
    
    n_fft = 2048 # windowsize
    hop_length = int(np.floor(n_fft / 4))
    
    # speedup
    stft = torch.stft(sample, n_fft, hop_length=hop_length, window=window).unsqueeze(0)
    phase_advance = torch.linspace(0, math.pi * hop_length, stft.shape[1])[..., None]
    vocoded = AF.phase_vocoder(stft, rate=speedup_rate, phase_advance=phase_advance)
    istft = AF.istft(vocoded, n_fft, hop_length=hop_length, window=window).squeeze()
    
    max_length = sample.shape[0]

    if speedup_rate > 1: 
        # faster means output is smaller -> padding
        pad_l = int((max_length - istft.shape[0])/2)
        pad_r = max_length - (pad_l + istft.shape[0])
        return F.pad(istft, (pad_l, pad_r))
    else:
        # slower means longer -> chopping of
        low = int((istft.shape[0] - max_length)/2)
        return istft[low:low+max_length]

In [84]:
def speed_attack(model, x, y, epsilon, num_iter=1):
    a=0.9
    b=1.1
    rate_search_range = torch.arange(a, b, float(1)/num_iter)
    losses = []
    stretched_inputs = []
    
    with torch.no_grad():
        for rate in rate_search_range:
            stretched = time_stretch(x.squeeze().cpu(), rate).cuda().unsqueeze(0)
            stretched_inputs.append(stretched)
            losses.append(F.nll_loss(model(stretched), y))
            
    best_rate = torch.stack(losses).argmax().item()
    return stretched_inputs[best_rate]

In [66]:
torch.utils.data.DataLoader(audioset)

In [82]:
valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)
acc, ex = testAttack(model, valid_loader, speed_attack, -1, num_iter=20, early_stopping=-1)

 43%|████▎     | 732/1687 [38:56<50:48,  3.19s/it]  


KeyboardInterrupt: 

In [79]:
eps = -1
adversarial_samples = {}

for sample in ex:
    if sample[1] == 1: # ie. previously correctly classified as EM
        print("found good adversarial sample: ")
        example_id = sample[0]
        print("\tSample id: " + str(example_id))
        adversarial_samples.update({example_id : sample})

found good adversarial sample: 
	Sample id: 7


In [ ]:
sample = adversarial_samples[7] #random.sample(list(adversarial_samples.values()),1)[0]
print(sample[0])
original = validation_dataset[sample[0]][0]
adversarial = sample[-1]

ipd.display(ipd.Audio(original,    rate=FIXED_SAMPLE_RATE, normalize=False))
ipd.display(ipd.Audio(adversarial, rate=FIXED_SAMPLE_RATE, normalize=False))

In [ ]:
raise Exception("hi")

## Interpolation attack

In [294]:
!git pull

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.
From gitlab.lrz.de:ml-lab-summer-2020/project-4
   f477cd8..8dba3af  develop_yan -> origin/develop_yan
Updating f477cd8..8dba3af
Fast-forward
 src/attacks/interpolation.py | 1 +
 1 file changed, 1 insertion(+)


In [284]:
path_tum_sound = '/nfs/students/summer-term-2020/project-4/yan/tum.wav'
tum_sound,sr = librosa.load(path_tum_sound, sr=FIXED_SAMPLE_RATE)
padding = int((max_length - len(tum_sound))/2)
zero_padded_data = torch.zeros(max_length)
zero_padded_data[padding:padding+tum_sound.shape[0]] = torch.from_numpy(tum_sound)
tum_sound = zero_padded_data

In [295]:
from attacks.interpolation import InterpolationAttack

valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)
attack_parameters = {'epsilon': 0.2, 'num_iter': 30, 'overlay_sound': tum_sound}

ia = InterpolationAttack(model, valid_loader, attack_parameters, early_stopping=1)
ia.attack()

  4%|▍         | 74/1687 [02:04<45:03,  1.68s/it]

Early stopping


In [298]:
# ia.showAdversarialExample(sr=FIXED_SAMPLE_RATE, target_class=1)

In [297]:
ia.report()

Attack-Parameters:	{'epsilon': 0.2, 'num_iter': 30, 'overlay_sound': tensor([0., 0., 0.,  ..., 0., 0., 0.])}
Early stopping: 	True (1)

Successfully attacked:	1
Total attacked: 	67
Total processed:	75

Success-Rate: 		0.01
Perturbed Accurracy: 	0.88


## Volume attacks

In [287]:
from attacks.volume import VolumeAttack

valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)
attack_parameters = {'epsilon': 0.2, 'num_iter': 30}

vol = VolumeAttack(model, valid_loader, attack_parameters, early_stopping=1)
vol.attack()

  0%|          | 7/1687 [00:14<58:05,  2.07s/it]

Early stopping


In [282]:
# vol.showAdversarialExample(sr=FIXED_SAMPLE_RATE, target_class=1)

In [283]:
vol.report()

Attack-Parameters:	{'epsilon': 0.2, 'num_iter': 30}
Early stopping: 	True (1)

Successfully attacked:	1
Total attacked: 	8
Total processed:	8

Success-Rate: 		0.12
Perturbed Accurracy: 	0.88


## FGSM

In [227]:
from attacks.fgsm import FGSM

valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)
attack_parameters = {'epsilon': 0.2}

fgsm = FGSM(model, valid_loader, attack_parameters, early_stopping=20)
fgsm.attack()

  3%|▎         | 53/1687 [00:05<02:35, 10.48it/s]

Early stopping


In [ ]:
# fgsm.showAdversarialExample(sr=FIXED_SAMPLE_RATE, target_class=1)

In [228]:
fgsm.report()

Attack-Parameters:	{'epsilon': 0.2}
Early stopping: 	True (20)

Successfully attacked:	20
Total attacked: 	49
Total processed:	54

Success-Rate: 		0.41
Perturbed Accurracy: 	0.54


## PGD

In [221]:
from attacks.pgd import PGD

valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)
attack_parameters = {'epsilon': 0.5, 'num_iter': 30}

pgd = PGD(model, valid_loader, attack_parameters, early_stopping=20)
pgd.attack()

  2%|▏         | 40/1687 [01:09<47:23,  1.73s/it]

Early stopping


In [ ]:
# pgd.showAdversarialExample(sr=FIXED_SAMPLE_RATE, target_class=1)

In [222]:
pgd.report()

Attack-Parameters:	{'epsilon': 0.5, 'num_iter': 30}
Early stopping: 	True (20)

Successfully attacked:	20
Total attacked: 	37
Total processed:	41

Success-Rate: 		0.54
Perturbed Accurracy: 	0.41
